In [6]:
import numpy as np                           #importing required libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from random import randrange
from itertools import islice

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
DATA_JSON_FILE = '/content/drive/MyDrive/News_Category_Dataset_v2.json'

In [9]:
data = pd.read_json(DATA_JSON_FILE, lines=True)
data.head(5)

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [10]:
target_category = data['category'].unique()
print(target_category)

['CRIME' 'ENTERTAINMENT' 'WORLD NEWS' 'IMPACT' 'POLITICS' 'WEIRD NEWS'
 'BLACK VOICES' 'WOMEN' 'COMEDY' 'QUEER VOICES' 'SPORTS' 'BUSINESS'
 'TRAVEL' 'MEDIA' 'TECH' 'RELIGION' 'SCIENCE' 'LATINO VOICES' 'EDUCATION'
 'COLLEGE' 'PARENTS' 'ARTS & CULTURE' 'STYLE' 'GREEN' 'TASTE'
 'HEALTHY LIVING' 'THE WORLDPOST' 'GOOD NEWS' 'WORLDPOST' 'FIFTY' 'ARTS'
 'WELLNESS' 'PARENTING' 'HOME & LIVING' 'STYLE & BEAUTY' 'DIVORCE'
 'WEDDINGS' 'FOOD & DRINK' 'MONEY' 'ENVIRONMENT' 'CULTURE & ARTS']


In [11]:
data['categoryId'] = data['category'].factorize()[0]
data.head()

,category,headline,authors,link,short_description,date,categoryId
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,0
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,1
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,1
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,1
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,1


In [12]:
category = data[['category', 'categoryId']].drop_duplicates().sort_values('categoryId')
category

,category,categoryId
0,CRIME,0
1,ENTERTAINMENT,1
11,WORLD NEWS,2
12,IMPACT,3
13,POLITICS,4
20,WEIRD NEWS,5
27,BLACK VOICES,6
35,WOMEN,7
37,COMEDY,8
69,QUEER VOICES,9


In [13]:
df = pd.DataFrame(data)
df.drop(df.iloc[:, 2::], inplace = True, axis = 1)

In [14]:
# After cleaning
df['headline'] = df['headline'].str.replace('\W', ' ') # Removes punctuation
df['headline'] = df['headline'].str.lower()
df.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,category,headline
0,CRIME,there were 2 mass shootings in texas last week...
1,ENTERTAINMENT,will smith joins diplo and nicky jam for the 2...
2,ENTERTAINMENT,hugh grant marries for the first time at age 57
3,ENTERTAINMENT,jim carrey blasts castrato adam schiff and d...
4,ENTERTAINMENT,julianna margulies uses donald trump poop bags...
5,ENTERTAINMENT,morgan freeman devastated that sexual harass...
6,ENTERTAINMENT,donald trump is lovin new mcdonald s jingle i...
7,ENTERTAINMENT,what to watch on amazon prime that s new this ...
8,ENTERTAINMENT,mike myers reveals he d like to do a fourth ...
9,ENTERTAINMENT,what to watch on hulu that s new this week


In [15]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    #stop_words=['those','is','and','a','at','these','for','these']
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')                                          #removing unnecessary characters
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("")

''

In [16]:
train_data = data[:int(0.9*len(data))]
test_and_dev_data = data[int(0.9*len(data)):]
                                                                       #splitting the data

dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180767 entries, 0 to 180766
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           180767 non-null  object        
 1   headline           180767 non-null  object        
 2   authors            180767 non-null  object        
 3   link               180767 non-null  object        
 4   short_description  180767 non-null  object        
 5   date               180767 non-null  datetime64[ns]
 6   categoryId         180767 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 9.7+ MB
None


In [17]:
train_data['category'].value_counts() 

POLITICS          32739
ENTERTAINMENT     15715
WELLNESS          13298
TRAVEL             7857
PARENTING          6748
HEALTHY LIVING     6694
QUEER VOICES       6030
STYLE & BEAUTY     5854
BUSINESS           5544
FOOD & DRINK       5280
COMEDY             4836
SPORTS             4778
BLACK VOICES       4314
PARENTS            3955
THE WORLDPOST      3664
WOMEN              3490
CRIME              3289
IMPACT             3252
HOME & LIVING      3196
MEDIA              2815
WEIRD NEWS         2670
WEDDINGS           2644
GREEN              2622
DIVORCE            2581
WORLDPOST          2579
RELIGION           2556
STYLE              2254
WORLD NEWS         2177
TASTE              2096
SCIENCE            1961
TECH               1815
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1129
ENVIRONMENT        1014
EDUCATION          1004
CULTURE & ARTS      835
MONEY               691
Name: category, 

In [18]:
X_train = train_data['headline']
y_train = train_data['category']
                                                    
X_dev = dev_data['headline']
y_dev = dev_data['category']


X_test = test_data['headline']
y_test = test_data['category']


In [25]:
train_category1 = train_data.loc[train_data.category == 'CRIME']
train_category2 = train_data.loc[train_data.category == 	'ENTERTAINMENT']
train_category3 = train_data.loc[train_data.category == 	'WORLD NEWS']
def cal_prior_probs(y_train):
  prior_Probability = {}
  for category in np.unique(y_train):
    categorycount = sum(y_train == category)
    prior_Probability[category] = categorycount / y_train.size
  return prior_Probability

prior_Probability = cal_prior_prob(y_train)


print("Total articles of category CRIME in train dataset : ",len(train_category1))
print("Total articles of category ENTERTAINMENT in train dataset : ",len(train_category2))
print("Total articles of category WORLD NEWS in train dataset : ",len(train_category3))

print("Prior probability in train article dataset of CRIME category: ",prior_Probability['CRIME'])
print("Prior probability in train article dataset of ENTERTAINMENT category : ",prior_Probability['ENTERTAINMENT'])
print("Prior probability in train article dataset of WORLD NEWS category: ",prior_Probability['WORLD NEWS'])

Total articles of category CRIME in train dataset :  3289
Total articles of category ENTERTAINMENT in train dataset :  15715
Total articles of category WORLD NEWS in train dataset :  2177
Prior probability in train article dataset of CRIME category:  0.018194692615355678
Prior probability in train article dataset of ENTERTAINMENT category :  0.08693511536950882
Prior probability in train article dataset of WORLD NEWS category:  0.012043127340720374
